In [41]:
import numpy as np
import pandas as pd
import torch
from allennlp.predictors.predictor import Predictor

In [53]:
predictor_small = Predictor.from_path("ser_dir_original_model/model.tar.gz")
predictor_large = Predictor.from_path("ser_dir_large_model/model.tar.gz")

Преобразуем данные из test.csv, парсим по колонкам

In [124]:
test_df = pd.read_csv('test.csv')[['Event','Xintent','Xemotion','Otheremotion']]
def str2list(x):
    return [y.strip() for y in x[2:-2].replace('"','').replace("'",'').split(',')]
for col in ['Xintent','Xemotion','Otheremotion']:
    test_df[col] = test_df[col].apply(str2list)

Группируем по фразам отдельно намерения, эмоции субъекта и эмоции окружающих

In [129]:
def clear_word(w):
    x = w
    if x[:3] == 'is ' or x[:3] == 'to ':
        x = x[3:]
    if x[:3] == 'be ':
        x = x[3:]
    if x[len(x) - 1] == '.':\
        x = x[:len(x)-1]
    return x

In [130]:
phrase2oemotion = {}
phrase2xemotion = {}
phrase2xintent = {}
res_xi = []
res_xe = []
res_oe = []
set_xi = set()
set_xe = set()
set_oe = set()

phrase_set = set([x for x in test_df.Event])

for phrase in phrase_set:
    for x in test_df[test_df.Event == phrase].Xintent:
        res_xi += x
    phrase2xintent[phrase] = set()
    for x in set(res_xi):
        if x != '':
            phrase2xintent[phrase].add(clear_word(x))
    res_xi.clear()
    
    for x in test_df[test_df.Event == phrase].Xemotion:
        res_xe += x
    phrase2xemotion[phrase] = set()
    for x in set(res_xe):
        if x != '':
            phrase2xemotion[phrase].add(clear_word(x))
    res_xe.clear()
    
    for x in test_df[test_df.Event == phrase].Otheremotion:
        res_oe += x
    phrase2oemotion[phrase] = set()
    for x in set(res_oe):
        if x != '':
            phrase2oemotion[phrase].add(clear_word(x))
    res_oe.clear()

In [131]:
p = "PersonX takes a nap"

In [132]:
phrase2xintent[p]

{'help a headache go away',
 'not be tired',
 'rest',
 'sleep',
 'take a break',
 'tired'}

In [133]:
result = predictor_large.predict(source=p)

oreact = set([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])    
intents = set([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
xreact = set([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

intents

{'get rest',
 'get some rest',
 'get to work',
 'have fun',
 'relax',
 'rest',
 'rest .',
 'rested',
 'sleep',
 'take rest'}

Считаем recall следующим образом: из всех вариантов, выданных моделью для данной фразы, оставляем те, что есть в контрольном множестве фразы, сформированном ранее, и делим их количество на размер контрольного множества. Затем делим полученную сумму на количество фраз. Проделываем для каждого из выходов (intent, xreact и oreact)

In [134]:
res_oe = 0
res_xe = 0
res_xi = 0

for phrase in phrase_set:
    result = predictor_small.predict(source=phrase)

    oreact = set([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])    
    intents = set([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
    xreact = set([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

    res_oe += len(phrase2oemotion[phrase].intersection(oreact)) / len(phrase2oemotion[phrase])
    res_xe += len(phrase2xemotion[phrase].intersection(xreact)) / len(phrase2xemotion[phrase])
    res_xi += len(phrase2xintent[phrase].intersection(intents)) / len(phrase2xintent[phrase])

print("Recall for intents: ", res_xi / len(phrase_set))
print("Recall for x_react: ", res_xe / len(phrase_set))
print("Recall for o_react: ", res_oe / len(phrase_set))

Recall for intents:  0.2453708973629289
Recall for x_react:  0.25397246990872446
Recall for o_react:  0.6486898964986609


In [135]:
res_oe = 0
res_xe = 0
res_xi = 0

for phrase in phrase_set:
    result = predictor_large.predict(source=phrase)

    oreact = set([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])    
    intents = set([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
    xreact = set([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

    res_oe += len(phrase2oemotion[phrase].intersection(oreact)) / len(phrase2oemotion[phrase])
    res_xe += len(phrase2xemotion[phrase].intersection(xreact)) / len(phrase2xemotion[phrase])
    res_xi += len(phrase2xintent[phrase].intersection(intents)) / len(phrase2xintent[phrase])

print("Recall for intents: ", res_xi / len(phrase_set))
print("Recall for x_react: ", res_xe / len(phrase_set))
print("Recall for o_react: ", res_oe / len(phrase_set))

Recall for intents:  0.29091148632981323
Recall for x_react:  0.17863993760408028
Recall for o_react:  0.6410126689011149


In [136]:
res_oe = 0
res_xe = 0
res_xi = 0
count = 0

for phrase in phrase_set:
    result = predictor_large.predict(source=phrase)

    oreact = set([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])    
    intents = set([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
    xreact = set([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

    res_oe += len(phrase2oemotion[phrase].intersection(oreact)) / len(phrase2oemotion[phrase]) * test_df[test_df.Event == phrase].shape[0]
    res_xe += len(phrase2xemotion[phrase].intersection(xreact)) / len(phrase2xemotion[phrase]) * test_df[test_df.Event == phrase].shape[0]
    res_xi += len(phrase2xintent[phrase].intersection(intents)) / len(phrase2xintent[phrase]) * test_df[test_df.Event == phrase].shape[0]
    
    count += test_df[test_df.Event == phrase].shape[0]

print("Recall for intents: ", res_xi / count)
print("Recall for x_react: ", res_xe / count)
print("Recall for o_react: ", res_oe / count)

Recall for intents:  0.2910334637589951
Recall for x_react:  0.18088990432411264
Recall for o_react:  0.634825931905035
